In [15]:
try:
    import pandas as pd
    from pyedgar.utilities import edgarweb
except ModuleNotFoundError:
    print("Warning, pyedgar not found.")
    class _o_(object):
        def edgar_links(*args, **kwargs):
            return ''
    edgarweb = _o_()

print("linkhead(df, n=5, fields=None)")
def linkhead(df, n=5, fields=None):
    """
    Displays top rows of a dataframe, and includes
    links to the HTML and FTP websites if CIK and Accession are found.
    """
    w = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', -1)
    
    if fields is None:
        fields = list(df.columns)
    
    dfn = df.head(n).copy()
    
    if 'cik' in df.columns:
        dfn['cik'] = dfn['cik'].astype(int)
        dfn['links'] = dfn.apply(lambda row: edgarweb.edgar_links(row), axis=1)
        fields.append('links')
    
    html = dfn[fields].to_html(escape=False, index=False, na_rep="")
    
    display_html(html, raw=True)
    pd.set_option('display.max_colwidth', w)
    #return html

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print("timehist(dtseries_or_df, time_variable='year', y_tic_number=4, x_tic_skip=0)")
def timehist(dtseries_or_df, time_variable='year', y_tic_number=4, x_tic_skip=0):
    """
    Historgam of observations per time period
    """
    x_tic_skip += 1
    sns.set_style('darkgrid')
    sns.set_context('talk')
    _d = dtseries_or_df
    try:
        _d = _d.dt.__getattribute__(time_variable)
    except:
        try:
            _d = _d[time_variable]
        except:
            pass
    _g = _d.value_counts().sort_index()
    if len(_g) > 1000:
        logger.error("ERROR: You are trying to plot something with too many levels. Don't do that.")
        return 
    
    plt.figure(figsize=(13,2))
    ax = plt.gca()
    plt.bar(_g.index, _g, width=.9)
    
    # Format and label X axis
    ax.set_xlim(left=_g.index.min()-0.5, right=_g.index.max()+0.5)
    _t = _g.index[::x_tic_skip]
    plt.xticks(_t, map(str, _t), rotation='vertical')
    
    # Label Y Axis
    tene = math.log10(_g.max())//1-1
    topnum = math.ceil(_g.max() / 10**tene)
    ax.set_yticks([(topnum * i // y_tic_number)*10**tene for i in range(y_tic_number, 0, -1)])